# Topography

In [1]:
import os
import glob
arcpy.env.overwriteOutput = True
start_directory = r'T:\amandal2023$\Working'
dirs="DEMft"
hucname="NAME"
hucid='HUC12'
layout_name = "DEM"
# Use the glob library to search for .shp files in subdirectories
shp_files = glob.glob(os.path.join(start_directory, '**/StatePlane/*.shp'), recursive=True)
classbrks=[0,1,2,3,4,5,6,7,8,200]
labels=['Below 0','0-1','1-2','2-3','3-4','4-5','5-6','6-7','7-8','Above 8']

for shp_file in shp_files:
    directory, file_name = os.path.split(shp_file)
    ras_folder=directory.replace('Huc12\StatePlane','HucWise'+'\\'+dirs)
    with arcpy.da.SearchCursor(shp_file, ["SHAPE@", hucname,hucid]) as cursor:
        for row in cursor:
            # Define the output raster name based on the OID or any unique identifier
            ras_name = str(row[1].replace(' ','_'))+f'_{dirs}.tif'
            ras_file=os.path.join(ras_folder,ras_name)
            print(row[1])
            print(ras_file)
            # Define the path to your ArcGIS Pro project file (.aprx)
            project_path = r"T:\amandal2023$\Hialeah\ArcGIS_Projects\All_maps.aprx"

            # Load the ArcGIS Pro project
            aprx = arcpy.mp.ArcGISProject(project_path)

            # Access the layout named "DEM"

            layout = aprx.listLayouts(layout_name)[0]

            # List the layers in the layout
            layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
            boundary=layers_in_layout[0]
            raster=layers_in_layout[1]
            boundary_src=layers_in_layout[0].connectionProperties
            raster_src=layers_in_layout[1].connectionProperties
            new_conn_v = {'dataset':file_name,'workspace_factory': 'Shape File','connection_info':{'database': directory}}
            new_conn_r = {'dataset':ras_name,'workspace_factory': 'Raster','connection_info':{'database': ras_folder}}
            # Update the data source for layers_in_layout[0]
            layers_in_layout[0].updateConnectionProperties(boundary_src,new_conn_v)
            # Update the data source for layers_in_layout[1]
            layers_in_layout[1].updateConnectionProperties(raster_src, new_conn_r)
            sym = raster.symbology
            sym.colorizer.breakCount = len(classbrks)
            breaks=sym.colorizer.classBreaks
            i=0
            for brk in breaks:
                brk.upperBound = classbrks[i]
                brk.label =labels[i]
                i=i+1
            raster.symbology = sym
            # Define the definition query
            definition_query = f"{hucname} = '{row[1]}'"

            # Apply the definition query to the layer
            boundary.definitionQuery = definition_query
            # Zoom to the extent of all layers in the layout
            mf = layout.listElements("MAPFRAME_ELEMENT")[0]
            #mf.camera.setExtent(mf.getLayerExtent(raster, False, True))
            original_extent = row[0].extent
            mf.camera.setExtent(original_extent)
            if row[2]=='':
                # Rename the layers
                boundary.name = str(row[1])
            else:
                boundary.name = "HUC "+str(row[2])+" "+str(row[1])

            # Save the ArcGIS Pro project
            project_path=os.path.join(start_directory,layout_name+'.aprx')
            #aprx.saveACopy(project_path)
            map_folder=ras_folder.replace('HucWise','Maps\HucWise')
            # Make sure the output folder exists, or create it if it doesn't
            if not os.path.exists(map_folder):
                os.makedirs(map_folder)
            map_name=ras_name.replace('.tif','.jpg')
            map_file=os.path.join(map_folder,map_name)
            # Export the map layout to a JPEG
            layout.exportToJPEG(map_file, resolution=300)
            # Clean up by releasing the project
            del aprx

            
            
            
            
            

            

Hialeah city
T:\amandal2023$\Working\Hialeah\HucWise\DEMft\Hialeah_city_DEMft.tif
SNAKE CREEK CANAL
T:\amandal2023$\Working\Hialeah\HucWise\DEMft\SNAKE_CREEK_CANAL_DEMft.tif
LITTLE RIVER CANAL
T:\amandal2023$\Working\Hialeah\HucWise\DEMft\LITTLE_RIVER_CANAL_DEMft.tif
BISCAYNE CANAL
T:\amandal2023$\Working\Hialeah\HucWise\DEMft\BISCAYNE_CANAL_DEMft.tif
MIAMI RIVER
T:\amandal2023$\Working\Hialeah\HucWise\DEMft\MIAMI_RIVER_DEMft.tif
North Miami city
T:\amandal2023$\Working\NorthMiami\HucWise\DEMft\North_Miami_city_DEMft.tif
NORTH BISCAYNE BAY
T:\amandal2023$\Working\NorthMiami\HucWise\DEMft\NORTH_BISCAYNE_BAY_DEMft.tif
LITTLE RIVER CANAL
T:\amandal2023$\Working\NorthMiami\HucWise\DEMft\LITTLE_RIVER_CANAL_DEMft.tif
BISCAYNE CANAL
T:\amandal2023$\Working\NorthMiami\HucWise\DEMft\BISCAYNE_CANAL_DEMft.tif


# Water Table

In [4]:
import os
import glob
arcpy.env.overwriteOutput = True
start_directory = r'T:\amandal2023$\Working'
dirs="WTEft"
hucname="NAME"
layout_name = "WTE"
hucid='HUC12'
# Use the glob library to search for .shp files in subdirectories
shp_files = glob.glob(os.path.join(start_directory, '**/StatePlane/*.shp'), recursive=True)
classbrks=[2,4,6,8,10,100]
labels=['Below 2','2-4','4-6','6-8','8-10','Above 10']

for shp_file in shp_files:
    directory, file_name = os.path.split(shp_file)
    ras_folder=directory.replace('Huc12\StatePlane','HucWise'+'\\'+dirs)
    with arcpy.da.SearchCursor(shp_file, ["SHAPE@", hucname,hucid]) as cursor:
        for row in cursor:
            # Define the output raster name based on the OID or any unique identifier
            ras_name = str(row[1].replace(' ','_'))+f'_{dirs}.tif'
            ras_file=os.path.join(ras_folder,ras_name)
            print(row[1])
            print(ras_file)
            # Define the path to your ArcGIS Pro project file (.aprx)
            project_path = r"T:\amandal2023$\Hialeah\ArcGIS_Projects\All_maps.aprx"

            # Load the ArcGIS Pro project
            aprx = arcpy.mp.ArcGISProject(project_path)

            # Access the layout named "DEM"

            layout = aprx.listLayouts(layout_name)[0]

            # List the layers in the layout
            layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
            boundary=layers_in_layout[0]
            raster=layers_in_layout[1]
            boundary_src=layers_in_layout[0].connectionProperties
            raster_src=layers_in_layout[1].connectionProperties
            new_conn_v = {'dataset':file_name,'workspace_factory': 'Shape File','connection_info':{'database': directory}}
            new_conn_r = {'dataset':ras_name,'workspace_factory': 'Raster','connection_info':{'database': ras_folder}}
            # Update the data source for layers_in_layout[0]
            layers_in_layout[0].updateConnectionProperties(boundary_src,new_conn_v)
            # Update the data source for layers_in_layout[1]
            layers_in_layout[1].updateConnectionProperties(raster_src, new_conn_r)
            sym = raster.symbology
            sym.colorizer.breakCount = len(classbrks)
            breaks=sym.colorizer.classBreaks
            i=0
            for brk in breaks:
                brk.upperBound = classbrks[i]
                brk.label =labels[i]
                i=i+1
            raster.symbology = sym
            # Define the definition query
            definition_query = f"{hucname} = '{row[1]}'"

            # Apply the definition query to the layer
            boundary.definitionQuery = definition_query
            # Zoom to the extent of all layers in the layout
            mf = layout.listElements("MAPFRAME_ELEMENT")[0]
            #mf.camera.setExtent(mf.getLayerExtent(raster, False, True))
            original_extent = row[0].extent
            mf.camera.setExtent(original_extent)
            if row[2]=='':
                # Rename the layers
                boundary.name = str(row[1])
            else:
                boundary.name = "HUC "+str(row[2])+" "+str(row[1])

            # Save the ArcGIS Pro project
            project_path=os.path.join(start_directory,layout_name+'.aprx')
            #aprx.saveACopy(project_path)
            map_folder=ras_folder.replace('HucWise','Maps\HucWise')
            # Make sure the output folder exists, or create it if it doesn't
            if not os.path.exists(map_folder):
                os.makedirs(map_folder)
            map_name=ras_name.replace('.tif','.jpg')
            map_file=os.path.join(map_folder,map_name)
            # Export the map layout to a JPEG
            layout.exportToJPEG(map_file, resolution=300)
            # Clean up by releasing the project
            del aprx


Hialeah city
T:\amandal2023$\Working\Hialeah\HucWise\WTEft\Hialeah_city_WTEft.tif
SNAKE CREEK CANAL
T:\amandal2023$\Working\Hialeah\HucWise\WTEft\SNAKE_CREEK_CANAL_WTEft.tif
LITTLE RIVER CANAL
T:\amandal2023$\Working\Hialeah\HucWise\WTEft\LITTLE_RIVER_CANAL_WTEft.tif
BISCAYNE CANAL
T:\amandal2023$\Working\Hialeah\HucWise\WTEft\BISCAYNE_CANAL_WTEft.tif
MIAMI RIVER
T:\amandal2023$\Working\Hialeah\HucWise\WTEft\MIAMI_RIVER_WTEft.tif
North Miami city
T:\amandal2023$\Working\NorthMiami\HucWise\WTEft\North_Miami_city_WTEft.tif
NORTH BISCAYNE BAY
T:\amandal2023$\Working\NorthMiami\HucWise\WTEft\NORTH_BISCAYNE_BAY_WTEft.tif
LITTLE RIVER CANAL
T:\amandal2023$\Working\NorthMiami\HucWise\WTEft\LITTLE_RIVER_CANAL_WTEft.tif
BISCAYNE CANAL
T:\amandal2023$\Working\NorthMiami\HucWise\WTEft\BISCAYNE_CANAL_WTEft.tif


# Impervious

In [5]:
import os
import glob
arcpy.env.overwriteOutput = True
start_directory = r'T:\amandal2023$\Working'
dirs="ImperviousBinary"
# Access the layout 
layout_name = "Impervious"
hucname="NAME"
hucid='HUC12'
# Use the glob library to search for .shp files in subdirectories
shp_files = glob.glob(os.path.join(start_directory, '**/StatePlane/*.shp'), recursive=True)
for shp_file in shp_files:
    directory, file_name = os.path.split(shp_file)
    ras_folder=directory.replace('Huc12\StatePlane','HucWise'+'\\'+dirs)
    with arcpy.da.SearchCursor(shp_file, ["SHAPE@", hucname,hucid]) as cursor:
        for row in cursor:
            # Define the output raster name based on the OID or any unique identifier
            ras_name = str(row[1].replace(' ','_'))+f'_{dirs}.tif'
            ras_file=os.path.join(ras_folder,ras_name)
            print(row[1])
            print(ras_file)
            # Define the path to your ArcGIS Pro project file (.aprx)
            project_path = r"T:\amandal2023$\Hialeah\ArcGIS_Projects\All_maps.aprx"

            # Load the ArcGIS Pro project
            aprx = arcpy.mp.ArcGISProject(project_path)

            # Access the layout named "DEM"

            layout = aprx.listLayouts(layout_name)[0]

            # List the layers in the layout
            layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
            boundary=layers_in_layout[0]
            raster=layers_in_layout[1]
            boundary_src=layers_in_layout[0].connectionProperties
            raster_src=layers_in_layout[1].connectionProperties
            new_conn_v = {'dataset':file_name,'workspace_factory': 'Shape File','connection_info':{'database': directory}}
            new_conn_r = {'dataset':ras_name,'workspace_factory': 'Raster','connection_info':{'database': ras_folder}}
            # Update the data source for layers_in_layout[0]
            layers_in_layout[0].updateConnectionProperties(boundary_src,new_conn_v)
            # Update the data source for layers_in_layout[1]
            layers_in_layout[1].updateConnectionProperties(raster_src, new_conn_r)
            # Define the definition query
            definition_query = f"{hucname} = '{row[1]}'"

            # Apply the definition query to the layer
            boundary.definitionQuery = definition_query
            # Zoom to the extent of all layers in the layout
            mf = layout.listElements("MAPFRAME_ELEMENT")[0]
            #mf.camera.setExtent(mf.getLayerExtent(raster, False, True))
            original_extent = row[0].extent
            mf.camera.setExtent(original_extent)
            if row[2]=='':
                # Rename the layers
                boundary.name = str(row[1])
            else:
                boundary.name = "HUC "+str(row[2])+" "+str(row[1])

            # Save the ArcGIS Pro project
            project_path=os.path.join(start_directory,layout_name+'.aprx')
            #aprx.saveACopy(project_path)
            map_folder=ras_folder.replace('HucWise','Maps\HucWise')
            # Make sure the output folder exists, or create it if it doesn't
            if not os.path.exists(map_folder):
                os.makedirs(map_folder)
            map_name=ras_name.replace('.tif','.jpg')
            map_file=os.path.join(map_folder,map_name)
            # Export the map layout to a JPEG
            layout.exportToJPEG(map_file, resolution=300)
            # Clean up by releasing the project
            del aprx


Hialeah city
T:\amandal2023$\Working\Hialeah\HucWise\ImperviousBinary\Hialeah_city_ImperviousBinary.tif
SNAKE CREEK CANAL
T:\amandal2023$\Working\Hialeah\HucWise\ImperviousBinary\SNAKE_CREEK_CANAL_ImperviousBinary.tif
LITTLE RIVER CANAL
T:\amandal2023$\Working\Hialeah\HucWise\ImperviousBinary\LITTLE_RIVER_CANAL_ImperviousBinary.tif
BISCAYNE CANAL
T:\amandal2023$\Working\Hialeah\HucWise\ImperviousBinary\BISCAYNE_CANAL_ImperviousBinary.tif
MIAMI RIVER
T:\amandal2023$\Working\Hialeah\HucWise\ImperviousBinary\MIAMI_RIVER_ImperviousBinary.tif
North Miami city
T:\amandal2023$\Working\NorthMiami\HucWise\ImperviousBinary\North_Miami_city_ImperviousBinary.tif
NORTH BISCAYNE BAY
T:\amandal2023$\Working\NorthMiami\HucWise\ImperviousBinary\NORTH_BISCAYNE_BAY_ImperviousBinary.tif
LITTLE RIVER CANAL
T:\amandal2023$\Working\NorthMiami\HucWise\ImperviousBinary\LITTLE_RIVER_CANAL_ImperviousBinary.tif
BISCAYNE CANAL
T:\amandal2023$\Working\NorthMiami\HucWise\ImperviousBinary\BISCAYNE_CANAL_ImperviousBin

# Soil Storage

In [1]:
import os
import glob
arcpy.env.overwriteOutput = True
start_directory = r'T:\amandal2023$\Working'
dirs="SoilStorage"
# Access the layout 
layout_name = "SoilStorage"
hucname="NAME"
hucid='HUC12'
# Use the glob library to search for .shp files in subdirectories
shp_files = glob.glob(os.path.join(start_directory, '**/StatePlane/*.shp'), recursive=True)
classbrks=[0,1,2,3,4,100]
labels=['0, No storage','0-1','1-2','2-3','3-4','4+']

for shp_file in shp_files:
    directory, file_name = os.path.split(shp_file)
    ras_folder=directory.replace('Huc12\StatePlane','HucWise'+'\\'+dirs)
    with arcpy.da.SearchCursor(shp_file, ["SHAPE@", hucname,hucid]) as cursor:
        for row in cursor:
            # Define the output raster name based on the OID or any unique identifier
            ras_name = str(row[1].replace(' ','_'))+f'_{dirs}.tif'
            ras_file=os.path.join(ras_folder,ras_name)
            print(row[1])
            print(ras_file)
            # Define the path to your ArcGIS Pro project file (.aprx)
            project_path = r"T:\amandal2023$\Hialeah\ArcGIS_Projects\All_maps.aprx"

            # Load the ArcGIS Pro project
            aprx = arcpy.mp.ArcGISProject(project_path)

            # Access the layout named "DEM"

            layout = aprx.listLayouts(layout_name)[0]

            # List the layers in the layout
            layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
            boundary=layers_in_layout[0]
            raster=layers_in_layout[1]
            boundary_src=layers_in_layout[0].connectionProperties
            raster_src=layers_in_layout[1].connectionProperties
            new_conn_v = {'dataset':file_name,'workspace_factory': 'Shape File','connection_info':{'database': directory}}
            new_conn_r = {'dataset':ras_name,'workspace_factory': 'Raster','connection_info':{'database': ras_folder}}
            # Update the data source for layers_in_layout[0]
            layers_in_layout[0].updateConnectionProperties(boundary_src,new_conn_v)
            # Update the data source for layers_in_layout[1]
            layers_in_layout[1].updateConnectionProperties(raster_src, new_conn_r)
            sym = raster.symbology
            sym.colorizer.breakCount = len(classbrks)
            breaks=sym.colorizer.classBreaks
            i=0
            for brk in breaks:
                brk.upperBound = classbrks[i]
                brk.label =labels[i]
                i=i+1
            #raster.symbology = sym
            # Define the definition query
            definition_query = f"{hucname} = '{row[1]}'"

            # Apply the definition query to the layer
            boundary.definitionQuery = definition_query
            # Zoom to the extent of all layers in the layout
            mf = layout.listElements("MAPFRAME_ELEMENT")[0]
            #mf.camera.setExtent(mf.getLayerExtent(raster, False, True))
            original_extent = row[0].extent
            mf.camera.setExtent(original_extent)
            if row[2]=='':
                # Rename the layers
                boundary.name = str(row[1])
            else:
                boundary.name = "HUC "+str(row[2])+" "+str(row[1])

            # Save the ArcGIS Pro project
            project_path=os.path.join(start_directory,layout_name+'.aprx')
            #aprx.saveACopy(project_path)
            map_folder=ras_folder.replace('HucWise','Maps\HucWise')
            # Make sure the output folder exists, or create it if it doesn't
            if not os.path.exists(map_folder):
                os.makedirs(map_folder)
            map_name=ras_name.replace('.tif','.jpg')
            map_file=os.path.join(map_folder,map_name)
            # Export the map layout to a JPEG
            layout.exportToJPEG(map_file, resolution=300)
            # Clean up by releasing the project
            del aprx

Hialeah city
T:\amandal2023$\Working\Hialeah\HucWise\SoilStorage\Hialeah_city_SoilStorage.tif
SNAKE CREEK CANAL
T:\amandal2023$\Working\Hialeah\HucWise\SoilStorage\SNAKE_CREEK_CANAL_SoilStorage.tif
LITTLE RIVER CANAL
T:\amandal2023$\Working\Hialeah\HucWise\SoilStorage\LITTLE_RIVER_CANAL_SoilStorage.tif
BISCAYNE CANAL
T:\amandal2023$\Working\Hialeah\HucWise\SoilStorage\BISCAYNE_CANAL_SoilStorage.tif
MIAMI RIVER
T:\amandal2023$\Working\Hialeah\HucWise\SoilStorage\MIAMI_RIVER_SoilStorage.tif
North Miami city
T:\amandal2023$\Working\NorthMiami\HucWise\SoilStorage\North_Miami_city_SoilStorage.tif
NORTH BISCAYNE BAY
T:\amandal2023$\Working\NorthMiami\HucWise\SoilStorage\NORTH_BISCAYNE_BAY_SoilStorage.tif
LITTLE RIVER CANAL
T:\amandal2023$\Working\NorthMiami\HucWise\SoilStorage\LITTLE_RIVER_CANAL_SoilStorage.tif
BISCAYNE CANAL
T:\amandal2023$\Working\NorthMiami\HucWise\SoilStorage\BISCAYNE_CANAL_SoilStorage.tif


# UnsaturatedZone

In [7]:
import os
import glob
arcpy.env.overwriteOutput = True
start_directory = r'T:\amandal2023$\Working'
dirs="unsaturated"
# Access the layout 
layout_name = "UnsaturatedZone"
hucname="NAME"
hucid='HUC12'
# Use the glob library to search for .shp files in subdirectories
shp_files = glob.glob(os.path.join(start_directory, '**/StatePlane/*.shp'), recursive=True)
classbrks=[0,1,2,3,4,5,6,100]
labels=['0','0-1','1-2','2-3','3-4','4-5','5-6','6+']

for shp_file in shp_files:
    directory, file_name = os.path.split(shp_file)
    ras_folder=directory.replace('Huc12\StatePlane','HucWise'+'\\'+dirs)
    with arcpy.da.SearchCursor(shp_file, ["SHAPE@", hucname,hucid]) as cursor:
        for row in cursor:
            # Define the output raster name based on the OID or any unique identifier
            ras_name = str(row[1].replace(' ','_'))+f'_{dirs}.tif'
            ras_file=os.path.join(ras_folder,ras_name)
            print(row[1])
            print(ras_file)
            # Define the path to your ArcGIS Pro project file (.aprx)
            project_path = r"T:\amandal2023$\Hialeah\ArcGIS_Projects\All_maps.aprx"

            # Load the ArcGIS Pro project
            aprx = arcpy.mp.ArcGISProject(project_path)

            # Access the layout named "DEM"

            layout = aprx.listLayouts(layout_name)[0]

            # List the layers in the layout
            layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
            boundary=layers_in_layout[0]
            raster=layers_in_layout[1]
            boundary_src=layers_in_layout[0].connectionProperties
            raster_src=layers_in_layout[1].connectionProperties
            new_conn_v = {'dataset':file_name,'workspace_factory': 'Shape File','connection_info':{'database': directory}}
            new_conn_r = {'dataset':ras_name,'workspace_factory': 'Raster','connection_info':{'database': ras_folder}}
            # Update the data source for layers_in_layout[0]
            layers_in_layout[0].updateConnectionProperties(boundary_src,new_conn_v)
            # Update the data source for layers_in_layout[1]
            layers_in_layout[1].updateConnectionProperties(raster_src, new_conn_r)
            sym = raster.symbology
            sym.colorizer.breakCount = len(classbrks)
            breaks=sym.colorizer.classBreaks
            i=0
            for brk in breaks:
                brk.upperBound = classbrks[i]
                brk.label =labels[i]
                i=i+1
            raster.symbology = sym
            # Define the definition query
            definition_query = f"{hucname} = '{row[1]}'"

            # Apply the definition query to the layer
            boundary.definitionQuery = definition_query
            # Zoom to the extent of all layers in the layout
            mf = layout.listElements("MAPFRAME_ELEMENT")[0]
            #mf.camera.setExtent(mf.getLayerExtent(raster, False, True))
            original_extent = row[0].extent
            mf.camera.setExtent(original_extent)
            if row[2]=='':
                # Rename the layers
                boundary.name = str(row[1])
            else:
                boundary.name = "HUC "+str(row[2])+" "+str(row[1])

            # Save the ArcGIS Pro project
            project_path=os.path.join(start_directory,layout_name+'.aprx')
            #aprx.saveACopy(project_path)
            map_folder=ras_folder.replace('HucWise','Maps\HucWise')
            # Make sure the output folder exists, or create it if it doesn't
            if not os.path.exists(map_folder):
                os.makedirs(map_folder)
            map_name=ras_name.replace('.tif','.jpg')
            map_file=os.path.join(map_folder,map_name)
            # Export the map layout to a JPEG
            layout.exportToJPEG(map_file, resolution=300)
            # Clean up by releasing the project
            del aprx

Hialeah city
T:\amandal2023$\Working\Hialeah\HucWise\unsaturated\Hialeah_city_unsaturated.tif
SNAKE CREEK CANAL
T:\amandal2023$\Working\Hialeah\HucWise\unsaturated\SNAKE_CREEK_CANAL_unsaturated.tif
LITTLE RIVER CANAL
T:\amandal2023$\Working\Hialeah\HucWise\unsaturated\LITTLE_RIVER_CANAL_unsaturated.tif
BISCAYNE CANAL
T:\amandal2023$\Working\Hialeah\HucWise\unsaturated\BISCAYNE_CANAL_unsaturated.tif
MIAMI RIVER
T:\amandal2023$\Working\Hialeah\HucWise\unsaturated\MIAMI_RIVER_unsaturated.tif
North Miami city
T:\amandal2023$\Working\NorthMiami\HucWise\unsaturated\North_Miami_city_unsaturated.tif
NORTH BISCAYNE BAY
T:\amandal2023$\Working\NorthMiami\HucWise\unsaturated\NORTH_BISCAYNE_BAY_unsaturated.tif
LITTLE RIVER CANAL
T:\amandal2023$\Working\NorthMiami\HucWise\unsaturated\LITTLE_RIVER_CANAL_unsaturated.tif
BISCAYNE CANAL
T:\amandal2023$\Working\NorthMiami\HucWise\unsaturated\BISCAYNE_CANAL_unsaturated.tif


# WHC_Ratio

In [11]:
import os
import glob
arcpy.env.overwriteOutput = True
start_directory = r'T:\amandal2023$\Working'
dirs="WHC"
# Access the layout 
layout_name = "WHC_Ratio"
hucname="NAME"
hucid='HUC12'
# Use the glob library to search for .shp files in subdirectories
shp_files = glob.glob(os.path.join(start_directory, '**/StatePlane/*.shp'), recursive=True)
classbrks=[0,0.05,0.10,0.15,0.20,0.25,1]
labels=['0,No Capacity','0.00-0.05','0.05-0.10','0.10-0.15','0.15-0.20','0.20-0.25','0.25-1.00']

for shp_file in shp_files:
    directory, file_name = os.path.split(shp_file)
    ras_folder=directory.replace('Huc12\StatePlane','HucWise'+'\\'+dirs)
    with arcpy.da.SearchCursor(shp_file, ["SHAPE@", hucname,hucid]) as cursor:
        for row in cursor:
            # Define the output raster name based on the OID or any unique identifier
            ras_name = str(row[1].replace(' ','_'))+f'_{dirs}.tif'
            ras_file=os.path.join(ras_folder,ras_name)
            print(row[1])
            print(ras_file)
            # Define the path to your ArcGIS Pro project file (.aprx)
            project_path = r"T:\amandal2023$\Hialeah\ArcGIS_Projects\All_maps.aprx"

            # Load the ArcGIS Pro project
            aprx = arcpy.mp.ArcGISProject(project_path)

            # Access the layout named "DEM"

            layout = aprx.listLayouts(layout_name)[0]

            # List the layers in the layout
            layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
            boundary=layers_in_layout[0]
            raster=layers_in_layout[1]
            boundary_src=layers_in_layout[0].connectionProperties
            raster_src=layers_in_layout[1].connectionProperties
            new_conn_v = {'dataset':file_name,'workspace_factory': 'Shape File','connection_info':{'database': directory}}
            new_conn_r = {'dataset':ras_name,'workspace_factory': 'Raster','connection_info':{'database': ras_folder}}
            # Update the data source for layers_in_layout[0]
            layers_in_layout[0].updateConnectionProperties(boundary_src,new_conn_v)
            # Update the data source for layers_in_layout[1]
            layers_in_layout[1].updateConnectionProperties(raster_src, new_conn_r)
            sym = raster.symbology
            sym.colorizer.breakCount = len(classbrks)
            breaks=sym.colorizer.classBreaks
            i=0
            for brk in breaks:
                brk.upperBound = classbrks[i]
                brk.label =labels[i]
                i=i+1
            raster.symbology = sym
            # Define the definition query
            definition_query = f"{hucname} = '{row[1]}'"

            # Apply the definition query to the layer
            boundary.definitionQuery = definition_query
            # Zoom to the extent of all layers in the layout
            mf = layout.listElements("MAPFRAME_ELEMENT")[0]
            #mf.camera.setExtent(mf.getLayerExtent(raster, False, True))
            original_extent = row[0].extent
            mf.camera.setExtent(original_extent)
            if row[2]=='':
                # Rename the layers
                boundary.name = str(row[1])
            else:
                boundary.name = "HUC "+str(row[2])+" "+str(row[1])

            # Save the ArcGIS Pro project
            project_path=os.path.join(start_directory,layout_name+'.aprx')
            #aprx.saveACopy(project_path)
            map_folder=ras_folder.replace('HucWise','Maps\HucWise')
            # Make sure the output folder exists, or create it if it doesn't
            if not os.path.exists(map_folder):
                os.makedirs(map_folder)
            map_name=ras_name.replace('.tif','.jpg')
            map_file=os.path.join(map_folder,map_name)
            # Export the map layout to a JPEG
            layout.exportToJPEG(map_file, resolution=300)
            # Clean up by releasing the project
            del aprx

Hialeah city
T:\amandal2023$\Working\Hialeah\HucWise\WHC\Hialeah_city_WHC.tif
SNAKE CREEK CANAL
T:\amandal2023$\Working\Hialeah\HucWise\WHC\SNAKE_CREEK_CANAL_WHC.tif
LITTLE RIVER CANAL
T:\amandal2023$\Working\Hialeah\HucWise\WHC\LITTLE_RIVER_CANAL_WHC.tif
BISCAYNE CANAL
T:\amandal2023$\Working\Hialeah\HucWise\WHC\BISCAYNE_CANAL_WHC.tif
MIAMI RIVER
T:\amandal2023$\Working\Hialeah\HucWise\WHC\MIAMI_RIVER_WHC.tif
North Miami city
T:\amandal2023$\Working\NorthMiami\HucWise\WHC\North_Miami_city_WHC.tif
NORTH BISCAYNE BAY
T:\amandal2023$\Working\NorthMiami\HucWise\WHC\NORTH_BISCAYNE_BAY_WHC.tif
LITTLE RIVER CANAL
T:\amandal2023$\Working\NorthMiami\HucWise\WHC\LITTLE_RIVER_CANAL_WHC.tif
BISCAYNE CANAL
T:\amandal2023$\Working\NorthMiami\HucWise\WHC\BISCAYNE_CANAL_WHC.tif


# Water

In [38]:
import os
import glob
arcpy.env.overwriteOutput = True
start_directory = r'T:\amandal2023$\Working'
dirs="WaterBinary"
# Access the layout 
layout_name = "Water"
hucname="NAME"
hucid='HUC12'
# Use the glob library to search for .shp files in subdirectories
shp_files = glob.glob(os.path.join(start_directory, '**/StatePlane/*.shp'), recursive=True)
for shp_file in shp_files:
    directory, file_name = os.path.split(shp_file)
    ras_folder=directory.replace('Huc12\StatePlane','HucWise'+'\\'+dirs)
    with arcpy.da.SearchCursor(shp_file, ["SHAPE@", hucname,hucid]) as cursor:
        for row in cursor:
            # Define the output raster name based on the OID or any unique identifier
            ras_name = str(row[1].replace(' ','_'))+f'_{dirs}.tif'
            ras_file=os.path.join(ras_folder,ras_name)
            print(row[1])
            print(ras_file)
            # Define the path to your ArcGIS Pro project file (.aprx)
            project_path = r"T:\amandal2023$\Hialeah\ArcGIS_Projects\All_maps.aprx"

            # Load the ArcGIS Pro project
            aprx = arcpy.mp.ArcGISProject(project_path)

            # Access the layout named "DEM"

            layout = aprx.listLayouts(layout_name)[0]

            # List the layers in the layout
            layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
            boundary=layers_in_layout[0]
            raster=layers_in_layout[1]
            boundary_src=layers_in_layout[0].connectionProperties
            raster_src=layers_in_layout[1].connectionProperties
            new_conn_v = {'dataset':file_name,'workspace_factory': 'Shape File','connection_info':{'database': directory}}
            new_conn_r = {'dataset':ras_name,'workspace_factory': 'Raster','connection_info':{'database': ras_folder}}
            # Update the data source for layers_in_layout[0]
            layers_in_layout[0].updateConnectionProperties(boundary_src,new_conn_v)
            # Update the data source for layers_in_layout[1]
            layers_in_layout[1].updateConnectionProperties(raster_src, new_conn_r)

            # Define the definition query
            definition_query = f"{hucname} = '{row[1]}'"

            # Apply the definition query to the layer
            boundary.definitionQuery = definition_query
            # Zoom to the extent of all layers in the layout
            mf = layout.listElements("MAPFRAME_ELEMENT")[0]
            #mf.camera.setExtent(mf.getLayerExtent(raster, False, True))
            original_extent = row[0].extent
            mf.camera.setExtent(original_extent)
            if row[2]=='':
                # Rename the layers
                boundary.name = str(row[1])
            else:
                boundary.name = "HUC "+str(row[2])+" "+str(row[1])

            # Save the ArcGIS Pro project
            project_path=os.path.join(start_directory,layout_name+'.aprx')
            #aprx.saveACopy(project_path)
            map_folder=ras_folder.replace('HucWise','Maps\HucWise')
            # Make sure the output folder exists, or create it if it doesn't
            if not os.path.exists(map_folder):
                os.makedirs(map_folder)
            map_name=ras_name.replace('.tif','.jpg')
            map_file=os.path.join(map_folder,map_name)
            # Export the map layout to a JPEG
            layout.exportToJPEG(map_file, resolution=300)
            # Clean up by releasing the project
            del aprx

Hialeah city
T:\amandal2023$\Working\Hialeah\HucWise\WaterBinary\Hialeah_city_WaterBinary.tif


KeyboardInterrupt: 

# Rainfall

In [23]:
import os
import glob
arcpy.env.overwriteOutput = True
start_directory = r'T:\amandal2023$\Working'
#dirs="WaterBinary"
# Access the layout 
layout_name = "Rainfall"
hucname="NAME"
hucid='HUC12'
dirlist=['r1d5yr','r1d10yr','r1d100yr','r3d25yr']
#dirlist=['r1d5yr']
labeldict={'1-Day 5-Year Rainfall':'Rainfall, 1-Day 5-Year \nStorm Event','1-Day 10-Year Rainfall':'Rainfall, 1-Day 10-Year \nStorm Event',
          '1-Day 100-Year Rainfall':'Rainfall, 1-Day 100-Year \nStorm Event','3-Day 25-Year Rainfall':'Rainfall, 3-Day 25-Year \nStorm Event'}
# Convert the dictionary to a list of key-value pairs (tuples)
dict_items = list(labeldict.items())
i=0
for dirs in dirlist:
    key, value = dict_items[i]
    # Use the glob library to search for .shp files in subdirectories
    shp_files = glob.glob(os.path.join(start_directory, '**/StatePlane/*.shp'), recursive=True)
    for shp_file in shp_files:
        directory, file_name = os.path.split(shp_file)
        ras_folder=directory.replace('Huc12\StatePlane','HucWise'+'\\'+dirs)
        with arcpy.da.SearchCursor(shp_file, ["SHAPE@", hucname,hucid]) as cursor:
            for row in cursor:
                # Define the output raster name based on the OID or any unique identifier
                ras_name = str(row[1].replace(' ','_'))+f'_{dirs}.tif'
                ras_file=os.path.join(ras_folder,ras_name)
                print(row[1])
                print(ras_file)
                # Define the path to your ArcGIS Pro project file (.aprx)
                project_path = r"T:\amandal2023$\Hialeah\ArcGIS_Projects\All_maps.aprx"

                # Load the ArcGIS Pro project
                aprx = arcpy.mp.ArcGISProject(project_path)

                # Access the layout named "DEM"

                layout = aprx.listLayouts(layout_name)[0]
                title=layout.listElements("TEXT_ELEMENT")[1]

                # List the layers in the layout
                layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
                boundary=layers_in_layout[0]
                raster=layers_in_layout[1]
                boundary_src=layers_in_layout[0].connectionProperties
                raster_src=layers_in_layout[1].connectionProperties
                new_conn_v = {'dataset':file_name,'workspace_factory': 'Shape File','connection_info':{'database': directory}}
                new_conn_r = {'dataset':ras_name,'workspace_factory': 'Raster','connection_info':{'database': ras_folder}}
                # Update the data source for layers_in_layout[0]
                layers_in_layout[0].updateConnectionProperties(boundary_src,new_conn_v)
                # Update the data source for layers_in_layout[1]
                layers_in_layout[1].updateConnectionProperties(raster_src, new_conn_r)
                sym = raster.symbology
                sym.colorizer.breakCount = 12
                sym.colorizer.classificationField="Inches"
                # Round up the upper bounds of class breaks to two decimal places and update labels
                for j in range(sym.colorizer.breakCount):
                    upper_bound = round(sym.colorizer.classBreaks[j].upperBound, 2)
                    sym.colorizer.classBreaks[j].upperBound = upper_bound
                    #print(sym.colorizer.classBreaks[i].label)
                    b=sym.colorizer.classBreaks[j].label
                    parts = b.split(" - ")
                    # Round and convert each part to a float
                    rounded_numbers = [round(float(part), 2) for part in parts]
                    # Convert the rounded floats back to strings
                    rounded_strings = [f"{rounded:.2f}" for rounded in rounded_numbers]
                    # Create a new formatted string
                    brk_label = " - ".join(rounded_strings)
                    sym.colorizer.classBreaks[j].label = brk_label
                    #print(sym.colorizer.classBreaks[i].label)
                raster.symbology = sym
                # Define the definition query
                definition_query = f"{hucname} = '{row[1]}'"

                # Apply the definition query to the layer
                boundary.definitionQuery = definition_query
                # Zoom to the extent of all layers in the layout
                mf = layout.listElements("MAPFRAME_ELEMENT")[0]
                #mf.camera.setExtent(mf.getLayerExtent(raster, False, True))
                original_extent = row[0].extent
                mf.camera.setExtent(original_extent)
                if row[2]=='':
                    # Rename the layers
                    boundary.name = str(row[1])
                else:
                    boundary.name = "HUC "+str(row[2])+"\n"+str(row[1])
                raster.name=key
                title.text=value

                # Save the ArcGIS Pro project
                project_path=os.path.join(start_directory,layout_name+'.aprx')
                #aprx.saveACopy(project_path)
                map_folder=ras_folder.replace('HucWise','Maps\HucWise')
                # Make sure the output folder exists, or create it if it doesn't
                if not os.path.exists(map_folder):
                    os.makedirs(map_folder)
                map_name=ras_name.replace('.tif','.jpg')
                map_file=os.path.join(map_folder,map_name)
                # Export the map layout to a JPEG
                layout.exportToJPEG(map_file, resolution=300)
                # Clean up by releasing the project
                del aprx
    i=i+1

Hialeah city
T:\amandal2023$\Working\Hialeah\HucWise\r1d5yr\Hialeah_city_r1d5yr.tif
SNAKE CREEK CANAL
T:\amandal2023$\Working\Hialeah\HucWise\r1d5yr\SNAKE_CREEK_CANAL_r1d5yr.tif
LITTLE RIVER CANAL
T:\amandal2023$\Working\Hialeah\HucWise\r1d5yr\LITTLE_RIVER_CANAL_r1d5yr.tif
BISCAYNE CANAL
T:\amandal2023$\Working\Hialeah\HucWise\r1d5yr\BISCAYNE_CANAL_r1d5yr.tif
MIAMI RIVER
T:\amandal2023$\Working\Hialeah\HucWise\r1d5yr\MIAMI_RIVER_r1d5yr.tif
North Miami city
T:\amandal2023$\Working\NorthMiami\HucWise\r1d5yr\North_Miami_city_r1d5yr.tif
NORTH BISCAYNE BAY
T:\amandal2023$\Working\NorthMiami\HucWise\r1d5yr\NORTH_BISCAYNE_BAY_r1d5yr.tif
LITTLE RIVER CANAL
T:\amandal2023$\Working\NorthMiami\HucWise\r1d5yr\LITTLE_RIVER_CANAL_r1d5yr.tif
BISCAYNE CANAL
T:\amandal2023$\Working\NorthMiami\HucWise\r1d5yr\BISCAYNE_CANAL_r1d5yr.tif
Hialeah city
T:\amandal2023$\Working\Hialeah\HucWise\r1d10yr\Hialeah_city_r1d10yr.tif
SNAKE CREEK CANAL
T:\amandal2023$\Working\Hialeah\HucWise\r1d10yr\SNAKE_CREEK_CANAL_r1

# Inundation

In [1]:
def lname(strng):
    scenario=strng
    # Define the regular expressions for the patterns you want to extract
    pattern1 = r'(\d+d\d+yr)'
    pattern2 = r'(\d+ft)'
    pattern3 = r'(kt)'
    pattern4 = r'(pump)'

    # Use re.findall to find all occurrences of the patterns in the input string
    matches1 = re.findall(pattern1, scenario)
    matches2 = re.findall(pattern2, scenario)
    matches3 = re.findall(pattern3, scenario)
    matches4 = re.findall(pattern4, scenario)
    #strom=(matches1[0].replace('d','-day ')).replace('yr','-year ') +'storm event'
    #slr=matches2[0]+' sea level rise and '
    #kt='2.6ft King Tides'
    #rint(matches1)
    d,y=matches1[0].split('d')
    #print(d)

    # Extracted substrings
    #substring1 = d.replace('d','-day ')+str(y)+'-year ' +'storm event' if matches1 else ''
    substring1 = str(d)+'-day '+y.replace('yr','-year ') +'storm event' if matches1 else ''
    substring2 = matches2[0].replace('ft','')+'-feet sea level rise' if matches2 else ''
    substring3 = '2.6ft King Tides' if matches3 else ''
    substring4 = 'after installing pumps at outlets' if matches4 else ''
    #rint(substring4)
     # Determine the lname variable based on the presence of substrings
    #for kt
    if substring1 == substring2 ==substring4=='' and substring3 != '':
        print('case1')
        lname = 'Probability of inundation based on ' + substring3
    #for %d%y
    elif substring4==substring3 == substring2 == '' and substring1 != '':
        print('case2')
        lname = 'Probability of inundation based on the ' + substring1
    
    #for %d%y+SLR
    elif substring1 != '' and substring2 != '' and substring3 == substring4=='':
        print('case3')
        lname = 'Probability of inundation based on ' + substring2 + ' and ' + substring1
    #for %d%y+SLR+pump
    elif substring4 !='':
        print('case4')
        lname = 'Probability of inundation based on ' + substring2 + ' and ' + substring1 + ' '+substring4
   
    #for %d%y+kt
    elif substring1 != '' and substring4==substring2 == '' and substring3 != '':
        print('case5')
        lname = 'Probability of inundation based on ' + substring3 + ' and ' + substring1
    
    #for %d%y+SLR+kt
    else:
        print('case6')
        lname = 'Probability of inundation based on ' +substring3 +','+ substring2 + ' and ' + substring1

    return lname
def round_scale(number):
    if number<100000:
        return (int(number / 1000) * 1000)+2000
    else:
        return (int(number / 1000) * 1000)+5000
    
def gethid(shp_file,huc):
    hid=''
    with arcpy.da.SearchCursor(shp_file, ["SHAPE@", hucname,hucid]) as cursor:
        for row in cursor:
            if row[1]==huc:
                hid=row[2]
    return hid

In [7]:
import os
import glob,re
#import textwrap
arcpy.env.overwriteOutput = True
start_directory = r'X:\amandal2023$\Working\NorthMiami'
dirs="Zscore_FloodRisk"
ras_folder=r'X:\amandal2023$\Working\NorthMiami\HucWise\Datasets\Inundation'
# Access the layout 
layout_name = "Zscore_FloodRisk"
hucname="NAME"
hucid='HUC12'
exlist=['HIALEAH HUC12s','NORTH MIAMI HUC12s']
# Use the glob library to search for .shp files in subdirectories
shp_files = glob.glob(os.path.join(start_directory, '**/StatePlane/*HUC12s.shp'), recursive=True)
for shp_file in shp_files:
    directory, file_name = os.path.split(shp_file)
    print(shp_file)
    #ras_folder=directory.replace('Huc12\StatePlane','HucWise'+'\\'+dirs)
    water_binary=directory.replace('Huc12\StatePlane','HucWise\Datasets\WaterBinary')
    inun_files=glob.glob(os.path.join(ras_folder, '**/*.tif'), recursive=True)
    for inun_file in inun_files:
        print(inun_file)
        # Split the file path into directory and file name
        directory1, file_name1 = os.path.split(inun_file)
        scenario=directory1.split('\\')[-1]
        print(scenario)
        waterlayername=file_name1.replace(scenario,'WaterBinary')
        waterlayerfile=os.path.join(water_binary,waterlayername)
        # Define the path to your ArcGIS Pro project file (.aprx)
        #project_path = r"T:\amandal2023$\Hialeah\ArcGIS_Projects\All_maps.aprx"

        # Load the ArcGIS Pro project
        aprx = arcpy.mp.ArcGISProject("current")

        # Access the layout named "DEM"

        layout = aprx.listLayouts(layout_name)[0]
        # List the layers in the layout
        layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
        boundary=layers_in_layout[1]
        smlyr=layers_in_layout[2]
        water=layers_in_layout[3]
        flood=layers_in_layout[4]
        boundary_src=boundary.connectionProperties
        water_src=water.connectionProperties
        flood_src=flood.connectionProperties
        new_conn_v = {'dataset':file_name,'workspace_factory': 'Shape File','connection_info':{'database': directory}}
        new_conn_r = {'dataset':waterlayername,'workspace_factory': 'Raster','connection_info':{'database': water_binary}}
        new_conn_f = {'dataset':file_name1,'workspace_factory': 'Raster','connection_info':{'database': directory1}}
        # Update the data source for layers_in_layout[0]
        layers_in_layout[1].updateConnectionProperties(boundary_src,new_conn_v)
        # Update the data source for layers_in_layout[1]
        water.updateConnectionProperties(water_src, new_conn_r)
        flood.updateConnectionProperties(flood_src, new_conn_f)
        qval=(file_name1.replace('_'+scenario+'.tif','')).replace('_',' ')
        #flood.name=lname(scenario)
        smlyr.name=lname(scenario)
        print(smlyr)
        if qval in exlist:
            boundary.name=qval            
        else:
            # Define the definition query
            definition_query = f"{hucname} = '{qval}'"
            # Apply the definition query to the layer
            boundary.definitionQuery = definition_query
            hid=gethid(shp_file,qval)
            boundary.name='HUC '+hid+'\n'+qval
        # Zoom to the extent of all layers in the layout
        mf = layout.listElements("MAPFRAME_ELEMENT")[0]
        #mf.camera.setExtent(mf.getLayerExtent(raster, False, True))
        desc = arcpy.Describe(inun_file)
        extent = desc.extent

        #original_extent = boundary.extent
        mf.camera.setExtent(extent)
        # Get the map scale
        map_scale = mf.camera.scale
        rounded=round_scale(map_scale)
        mf.camera.scale=rounded
        #print(map_scale)
        #print(rounded)

        # Save the ArcGIS Pro project
        project_path=os.path.join(start_directory,layout_name+'.aprx')
        #aprx.saveACopy(project_path)
        map_folder=directory1.replace('Datasets','Maps')
        #X:\amandal2023$\Working\NorthMiami\HucWise\Maps\Inundation\1d100yr
        # Make sure the output folder exists, or create it if it doesn't
        if not os.path.exists(map_folder):
            os.makedirs(map_folder)
        map_name=file_name1.replace('.tif','.jpg')
        map_file=os.path.join(map_folder,map_name)
        # Export the map layout to a JPEG
        layout.exportToJPEG(map_file, resolution=300)
        # Clean up by releasing the project
        # Save the ArcGIS Pro project
        project_path=os.path.join(start_directory,layout_name+'.aprx')
        #aprx.saveACopy(project_path)
        boundary.definitionQuery = ""
        del aprx
    


X:\amandal2023$\Working\NorthMiami\Huc12\StatePlane\NORTH_MIAMI_HUC12s.shp
X:\amandal2023$\Working\NorthMiami\HucWise\Datasets\Inundation\1d100yr\BISCAYNE_CANAL_1d100yr.tif
1d100yr
case2
Probability of inundation based on the 1-day 100-year storm event
X:\amandal2023$\Working\NorthMiami\HucWise\Datasets\Inundation\1d100yr\LITTLE_RIVER_CANAL_1d100yr.tif
1d100yr
case2
Probability of inundation based on the 1-day 100-year storm event
X:\amandal2023$\Working\NorthMiami\HucWise\Datasets\Inundation\1d100yr\NORTH_BISCAYNE_BAY_1d100yr.tif
1d100yr
case2
Probability of inundation based on the 1-day 100-year storm event
X:\amandal2023$\Working\NorthMiami\HucWise\Datasets\Inundation\1d100yr\NORTH_MIAMI_HUC12s_1d100yr.tif
1d100yr
case2
Probability of inundation based on the 1-day 100-year storm event
X:\amandal2023$\Working\NorthMiami\HucWise\Datasets\Inundation\1d100yr_1ft\BISCAYNE_CANAL_1d100yr_1ft.tif
1d100yr_1ft
case3
Probability of inundation based on 1-feet sea level rise and 1-day 100-year s

Probability of inundation based on 3-feet sea level rise and 1-day 10-year storm event
X:\amandal2023$\Working\NorthMiami\HucWise\Datasets\Inundation\1d10yr_3ft\LITTLE_RIVER_CANAL_1d10yr_3ft.tif
1d10yr_3ft
case3
Probability of inundation based on 3-feet sea level rise and 1-day 10-year storm event
X:\amandal2023$\Working\NorthMiami\HucWise\Datasets\Inundation\1d10yr_3ft\NORTH_BISCAYNE_BAY_1d10yr_3ft.tif
1d10yr_3ft
case3
Probability of inundation based on 3-feet sea level rise and 1-day 10-year storm event
X:\amandal2023$\Working\NorthMiami\HucWise\Datasets\Inundation\1d10yr_3ft\NORTH_MIAMI_HUC12s_1d10yr_3ft.tif
1d10yr_3ft
case3
Probability of inundation based on 3-feet sea level rise and 1-day 10-year storm event
X:\amandal2023$\Working\NorthMiami\HucWise\Datasets\Inundation\1d10yr_3ft_kt\BISCAYNE_CANAL_1d10yr_3ft_kt.tif
1d10yr_3ft_kt
case6
Probability of inundation based on 2.6ft King Tides,3-feet sea level rise and 1-day 10-year storm event
X:\amandal2023$\Working\NorthMiami\HucWise\

X:\amandal2023$\Working\NorthMiami\HucWise\Datasets\Inundation\3d25yr\BISCAYNE_CANAL_3d25yr.tif
3d25yr
case2
Probability of inundation based on the 3-day 25-year storm event
X:\amandal2023$\Working\NorthMiami\HucWise\Datasets\Inundation\3d25yr\LITTLE_RIVER_CANAL_3d25yr.tif
3d25yr
case2
Probability of inundation based on the 3-day 25-year storm event
X:\amandal2023$\Working\NorthMiami\HucWise\Datasets\Inundation\3d25yr\NORTH_BISCAYNE_BAY_3d25yr.tif
3d25yr
case2
Probability of inundation based on the 3-day 25-year storm event
X:\amandal2023$\Working\NorthMiami\HucWise\Datasets\Inundation\3d25yr\NORTH_MIAMI_HUC12s_3d25yr.tif
3d25yr
case2
Probability of inundation based on the 3-day 25-year storm event
X:\amandal2023$\Working\NorthMiami\HucWise\Datasets\Inundation\3d25yr_1ft\BISCAYNE_CANAL_3d25yr_1ft.tif
3d25yr_1ft
case3
Probability of inundation based on 1-feet sea level rise and 3-day 25-year storm event
X:\amandal2023$\Working\NorthMiami\HucWise\Datasets\Inundation\3d25yr_1ft\LITTLE_RIV

IndexError: list index out of range

inundation santarosa

In [1]:
def lname(strng):
    # Define the regular expressions for the patterns you want to extract
    pattern1 = r'(\d+Y\d+D)'
    pattern2 = r'(S+\d)'
    pattern3 = r'(kt)'

    # Use re.findall to find all occurrences of the patterns in the input string
    matches1 = re.findall(pattern1, scenario)
    matches2 = re.findall(pattern2, scenario)
    matches3 = re.findall(pattern3, scenario)
    #strom=(matches1[0].replace('d','-day ')).replace('yr','-year ') +'storm event'
    #slr=matches2[0]+' sea level rise and '
    #kt='2.6ft King Tides'
    print(matches1)
    y,d=matches1[0].split('Y')

    # Extracted substrings
    substring1 = d.replace('D','-day ')+str(y)+'-year ' +'storm event' if matches1 else ''
    substring2 = matches2[0].replace('S','')+'ft sea level rise' if matches2 else ''
    substring3 = '2.6ft King Tides' if matches3 else ''
     # Determine the lname variable based on the presence of substrings
    #for kt
    if substring1 == substring2 == '' and substring3 != '':
        lname = 'Probability of inundation based on ' + substring3
    #for %d%y
    elif substring3 == substring2 == '' and substring1 != '':
        lname = 'Probability of inundation based on the ' + substring1
    
    #for %d%y+SLR
    elif substring1 != '' and substring2 != '' and substring3 == '':
        lname = 'Probability of inundation based on ' + substring2 + ' and ' + substring1
    #for %d%y+kt
    elif substring1 != '' and substring2 == '' and substring3 != '':
        lname = 'Probability of inundation based on ' + substring3 + ' and ' + substring1
    #for %d%y+SLR+kt
    else:
        lname = 'Probability of inundation based on ' +substring3 +','+ substring2 + ' and ' + substring1

    return lname
def round_scale(number):
    if number<100000:
        return (int(number / 1000) * 1000)+2000
    else:
        return (int(number / 1000) * 1000)+5000
    
def gethid(shp_file,huc):
    hid=''
    with arcpy.da.SearchCursor(shp_file, ["SHAPE@", hucname,hucid]) as cursor:
        for row in cursor:
            if row[1]==huc:
                hid=row[2]
    return hid

In [2]:
import os,re
import glob
arcpy.env.overwriteOutput = True
start_directory = r'T:\amandal2023$\Working\Santa Rosa\Datasets\ZScore\Mosaic'
map_folder=r'T:\amandal2023$\Working\Santa Rosa\Maps\Inundation'
dirs="Zscore_FloodRisk"
# Access the layout 
layout_name = "Zscore_FloodRisk"
inun_files=glob.glob(os.path.join(start_directory, '**/*.tif'), recursive=True)
for inun_file in inun_files:
    print(inun_file)
    # Split the file path into directory and file name
    directory1, file_name1 = os.path.split(inun_file)
    scenario=file_name1.split('.')[0]
    name=lname(scenario)
    print(name)
    # Define the path to your ArcGIS Pro project file (.aprx)
    #project_path = r"T:\amandal2023$\Hialeah\ArcGIS_Projects\All_maps.aprx"

    # Load the ArcGIS Pro project
    aprx = arcpy.mp.ArcGISProject('Current')

    # Access the layout named "DEM"

    layout = aprx.listLayouts(layout_name)[0]

    # List the layers in the layout
    layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
    #boundary=layers_in_layout[0]
    #water=layers_in_layout[1]
    flood=layers_in_layout[1]
    #boundary_src=layers_in_layout[0].connectionProperties
    #water_src=layers_in_layout[1].connectionProperties
    flood_src=layers_in_layout[1].connectionProperties
    #new_conn_v = {'dataset':file_name,'workspace_factory': 'Shape File','connection_info':{'database': directory}}
    #new_conn_r = {'dataset':waterlayername,'workspace_factory': 'Raster','connection_info':{'database': water_binary}}
    new_conn_f = {'dataset':file_name1,'workspace_factory': 'Raster','connection_info':{'database': directory1}}
    # Update the data source for layers_in_layout[0]
    #layers_in_layout[0].updateConnectionProperties(boundary_src,new_conn_v)
    # Update the data source for layers_in_layout[1]
    #layers_in_layout[1].updateConnectionProperties(water_src, new_conn_r)
    layers_in_layout[1].updateConnectionProperties(flood_src, new_conn_f)
    flood.name=lname(scenario)
    """qval=(file_name1.replace('_'+scenario+'.tif','')).replace('_',' ')
    flood.name=lname(scenario)
    if qval in exlist:
        boundary.name=qval
    else:
        # Define the definition query
        definition_query = f"{hucname} = '{qval}'"
        # Apply the definition query to the layer
        boundary.definitionQuery = definition_query
        hid=gethid(shp_file,qval)
        boundary.name='HUC '+hid+'\n'+qval
    # Zoom to the extent of all layers in the layout
    mf = layout.listElements("MAPFRAME_ELEMENT")[0]
    #mf.camera.setExtent(mf.getLayerExtent(raster, False, True))
    desc = arcpy.Describe(inun_file)
    extent = desc.extent

    #original_extent = boundary.extent
    mf.camera.setExtent(extent)
    # Get the map scale
    map_scale = mf.camera.scale
    rounded=round_scale(map_scale)
    mf.camera.scale=rounded
    print(map_scale)
    print(rounded)
    """

   
    # Make sure the output folder exists, or create it if it doesn't
    if not os.path.exists(map_folder):
        os.makedirs(map_folder)
    map_name=file_name1.replace('.tif','.jpg')
    map_file=os.path.join(map_folder,map_name)
    # Export the map layout to a JPEG
    layout.exportToJPEG(map_file, resolution=300)
    # Clean up by releasing the project
    
    del aprx


    

T:\amandal2023$\Working\Santa Rosa\Datasets\ZScore\Mosaic\100Y1D.tif
['100Y1D']
Probability of inundation based on the 1-day 100-year storm event
['100Y1D']
T:\amandal2023$\Working\Santa Rosa\Datasets\ZScore\Mosaic\100Y1D_S1.tif
['100Y1D']
Probability of inundation based on 1ft sea level rise and 1-day 100-year storm event
['100Y1D']
T:\amandal2023$\Working\Santa Rosa\Datasets\ZScore\Mosaic\100Y1D_S2.tif
['100Y1D']
Probability of inundation based on 2ft sea level rise and 1-day 100-year storm event
['100Y1D']
T:\amandal2023$\Working\Santa Rosa\Datasets\ZScore\Mosaic\100Y1D_S3.tif
['100Y1D']
Probability of inundation based on 3ft sea level rise and 1-day 100-year storm event
['100Y1D']
T:\amandal2023$\Working\Santa Rosa\Datasets\ZScore\Mosaic\100Y1D_S4.tif
['100Y1D']
Probability of inundation based on 4ft sea level rise and 1-day 100-year storm event
['100Y1D']
T:\amandal2023$\Working\Santa Rosa\Datasets\ZScore\Mosaic\100Y1D_S5.tif
['100Y1D']
Probability of inundation based on 5ft sea l

In [1]:
import os
import glob
arcpy.env.overwriteOutput = True
start_directory = r'T:\amandal2023$\Working'
dirs="Zscore_FloodRisk"
# Access the layout 
layout_name = "Zscore_FloodRisk"
hucname="NAME"
hucid='HUC12'
# Use the glob library to search for .shp files in subdirectories
shp_files = glob.glob(os.path.join(start_directory, '**/StatePlane/*HUC12.shp'), recursive=True)
for shp_file in shp_files:
    directory, file_name = os.path.split(shp_file)
    print(shp_file)
    ras_folder=directory.replace('Huc12\StatePlane','HucWise'+'\\'+'Inundation')
    water_binary=directory.replace('Huc12\StatePlane','HucWise'+'\\'+'WaterBinary')
    inun_files=glob.glob(os.path.join(ras_folder, '**/*.tif'), recursive=True)
    for inun_file in inun_files:
        print(inun_file)
        # Split the file path into directory and file name
        directory1, file_name1 = os.path.split(inun_file)
        scenario=directory1.split('\\')[-1]
        waterlayername=file_name1.replace(scenario,'WaterBinary')
        waterlayerfile=os.path.join(water_binary,waterlayername)
        if os.path.exists(waterlayerfile):
            print('exist')
            print(waterlayerfile)

T:\amandal2023$\Working\Hialeah\Huc12\StatePlane\Miami_Hialeah_City_HUC12.shp
T:\amandal2023$\Working\Hialeah\HucWise\Inundation\1d100yr\BISCAYNE_CANAL_1d100yr.tif
exist
T:\amandal2023$\Working\Hialeah\HucWise\WaterBinary\BISCAYNE_CANAL_WaterBinary.tif
T:\amandal2023$\Working\Hialeah\HucWise\Inundation\1d100yr\LITTLE_RIVER_CANAL_1d100yr.tif
exist
T:\amandal2023$\Working\Hialeah\HucWise\WaterBinary\LITTLE_RIVER_CANAL_WaterBinary.tif
T:\amandal2023$\Working\Hialeah\HucWise\Inundation\1d100yr\Miami_Hialeah_City_1d100yr.tif
exist
T:\amandal2023$\Working\Hialeah\HucWise\WaterBinary\Miami_Hialeah_City_WaterBinary.tif
T:\amandal2023$\Working\Hialeah\HucWise\Inundation\1d100yr\MIAMI_RIVER_1d100yr.tif
exist
T:\amandal2023$\Working\Hialeah\HucWise\WaterBinary\MIAMI_RIVER_WaterBinary.tif
T:\amandal2023$\Working\Hialeah\HucWise\Inundation\1d100yr\SNAKE_CREEK_CANAL_1d100yr.tif
exist
T:\amandal2023$\Working\Hialeah\HucWise\WaterBinary\SNAKE_CREEK_CANAL_WaterBinary.tif
T:\amandal2023$\Working\Hialeah

T:\amandal2023$\Working\Hialeah\HucWise\Inundation\1d10yr_3ft\Miami_Hialeah_City_1d10yr_3ft.tif
exist
T:\amandal2023$\Working\Hialeah\HucWise\WaterBinary\Miami_Hialeah_City_WaterBinary.tif
T:\amandal2023$\Working\Hialeah\HucWise\Inundation\1d10yr_3ft\MIAMI_RIVER_1d10yr_3ft.tif
exist
T:\amandal2023$\Working\Hialeah\HucWise\WaterBinary\MIAMI_RIVER_WaterBinary.tif
T:\amandal2023$\Working\Hialeah\HucWise\Inundation\1d10yr_3ft\SNAKE_CREEK_CANAL_1d10yr_3ft.tif
exist
T:\amandal2023$\Working\Hialeah\HucWise\WaterBinary\SNAKE_CREEK_CANAL_WaterBinary.tif
T:\amandal2023$\Working\Hialeah\HucWise\Inundation\1d10yr_3ft_kt\BISCAYNE_CANAL_1d10yr_3ft_kt.tif
exist
T:\amandal2023$\Working\Hialeah\HucWise\WaterBinary\BISCAYNE_CANAL_WaterBinary.tif
T:\amandal2023$\Working\Hialeah\HucWise\Inundation\1d10yr_3ft_kt\LITTLE_RIVER_CANAL_1d10yr_3ft_kt.tif
exist
T:\amandal2023$\Working\Hialeah\HucWise\WaterBinary\LITTLE_RIVER_CANAL_WaterBinary.tif
T:\amandal2023$\Working\Hialeah\HucWise\Inundation\1d10yr_3ft_kt\Mi

T:\amandal2023$\Working\Hialeah\HucWise\Inundation\1d5yr_kt\BISCAYNE_CANAL_1d5yr_kt.tif
exist
T:\amandal2023$\Working\Hialeah\HucWise\WaterBinary\BISCAYNE_CANAL_WaterBinary.tif
T:\amandal2023$\Working\Hialeah\HucWise\Inundation\1d5yr_kt\LITTLE_RIVER_CANAL_1d5yr_kt.tif
exist
T:\amandal2023$\Working\Hialeah\HucWise\WaterBinary\LITTLE_RIVER_CANAL_WaterBinary.tif
T:\amandal2023$\Working\Hialeah\HucWise\Inundation\1d5yr_kt\Miami_Hialeah_City_1d5yr_kt.tif
exist
T:\amandal2023$\Working\Hialeah\HucWise\WaterBinary\Miami_Hialeah_City_WaterBinary.tif
T:\amandal2023$\Working\Hialeah\HucWise\Inundation\1d5yr_kt\MIAMI_RIVER_1d5yr_kt.tif
exist
T:\amandal2023$\Working\Hialeah\HucWise\WaterBinary\MIAMI_RIVER_WaterBinary.tif
T:\amandal2023$\Working\Hialeah\HucWise\Inundation\1d5yr_kt\SNAKE_CREEK_CANAL_1d5yr_kt.tif
exist
T:\amandal2023$\Working\Hialeah\HucWise\WaterBinary\SNAKE_CREEK_CANAL_WaterBinary.tif
T:\amandal2023$\Working\Hialeah\HucWise\Inundation\3d25yr\BISCAYNE_CANAL_3d25yr.tif
exist
T:\amanda

T:\amandal2023$\Working\NorthMiami\HucWise\Inundation\1d100yr_4ft_kt\BISCAYNE_CANAL_1d100yr_4ft_kt.tif
exist
T:\amandal2023$\Working\NorthMiami\HucWise\WaterBinary\BISCAYNE_CANAL_WaterBinary.tif
T:\amandal2023$\Working\NorthMiami\HucWise\Inundation\1d100yr_4ft_kt\LITTLE_RIVER_CANAL_1d100yr_4ft_kt.tif
exist
T:\amandal2023$\Working\NorthMiami\HucWise\WaterBinary\LITTLE_RIVER_CANAL_WaterBinary.tif
T:\amandal2023$\Working\NorthMiami\HucWise\Inundation\1d100yr_4ft_kt\NorthMiami_1d100yr_4ft_kt.tif
exist
T:\amandal2023$\Working\NorthMiami\HucWise\WaterBinary\NorthMiami_WaterBinary.tif
T:\amandal2023$\Working\NorthMiami\HucWise\Inundation\1d100yr_4ft_kt\NORTH_BISCAYNE_BAY_1d100yr_4ft_kt.tif
exist
T:\amandal2023$\Working\NorthMiami\HucWise\WaterBinary\NORTH_BISCAYNE_BAY_WaterBinary.tif
T:\amandal2023$\Working\NorthMiami\HucWise\Inundation\1d100yr_5ft\BISCAYNE_CANAL_1d100yr_5ft.tif
exist
T:\amandal2023$\Working\NorthMiami\HucWise\WaterBinary\BISCAYNE_CANAL_WaterBinary.tif
T:\amandal2023$\Working

In [48]:
 # Define the path to your ArcGIS Pro project file (.aprx)
project_path = r"T:\amandal2023$\Hialeah\ArcGIS_Projects\All_maps.aprx"

# Load the ArcGIS Pro project
aprx = arcpy.mp.ArcGISProject(project_path)

# Access the layout named "DEM"

layout = aprx.listLayouts("Zscore_FloodRisk")[0]
title=layout.listElements("TEXT_ELEMENT")[1]
print(title.text)

for elm in layout.listElements("TEXT_ELEMENT"):
    print(elm.name)

Flood Inundation
Probability
Text 1
Text


In [51]:
print(layout.name)
# List the layers in the layout
layers_in_layout = layout.listElements("MAPFRAME_ELEMENT")[0].map.listLayers()
for layer in layers_in_layout:
    print(layer.name)

Zscore_FloodRisk
HUC 030902050102 Ninemile Canal
water
Probability of inundation based on the 3-day         25-year storm event and 5ft sea level rise
World Topographic Map
World Hillshade


In [3]:
# Define the path to your ArcGIS Pro project file (.aprx)
project_path = r"T:\amandal2023$\Hialeah\ArcGIS_Projects\All_maps.aprx"

# Load the ArcGIS Pro project
aprx = arcpy.mp.ArcGISProject(project_path)
layouts = aprx.listLayouts()
for layout in layouts:
    print(layout.name)

Zscore_FloodRisk
WTE
UnsaturatedZone
WHC_Ratio
Impervious
SoilStorage
Rainfall
Stations
StudyArea_HUC12s
Water
Stations_Drainage
FEMA_FIRM
DEM
SFWMD_LandUse
NLCD_LandCover
Infrastructure
